In [28]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import KFold

In [29]:
# Define the KNN class
class KNN:
    def __init__(self, k=3, distance_metric='euclidean'):
        self.k = k
        self.distance_metric = distance_metric

    def fit(self, X, y):
        # TODO: Implement the fit method
        self.X_train = X
        self.y_train = y

    def predict(self, X):
        # TODO: Implement the predict method

        predictions = []
        for x in X:
            distances = self.compute_distance(x, self.X_train)
            k_indices = np.argsort(distances)[:self.k]
            k_nearest_labels = self.y_train[k_indices]
            prediction = np.bincount(k_nearest_labels.astype(int)).argmax()
            predictions.append(prediction)
        return np.array(predictions)

    def compute_distance(self, X1, X2):
        # TODO: Implement distance computation based on self.distance_metric
        # Hint: Use numpy operations for efficient computation

        if self.distance_metric == 'euclidean':
            return np.sqrt(np.sum((X1 - X2)**2, axis=1))
        elif self.distance_metric == 'manhattan':
            return np.sum(np.abs(X1 - X2), axis=1)
        else:
            raise ValueError("Unsupported distance metric")

In [30]:
# Define data preprocessing function
def preprocess_data(train_path, test_path):
    train_data = pd.read_csv(train_path)
    test_data = pd.read_csv(test_path)

    # TODO: Implement data preprocessing
    # Handle categorical variables, scale features, etc.

    # Combine train and test for preprocessing
    all_data = pd.concat([train_data, test_data], axis=0)

    # Handle categorical variables
    categorical_cols = ['Geography', 'Gender']
    all_data = pd.get_dummies(all_data, columns=categorical_cols, drop_first=True)

    # Handle missing values
    # Only fill numeric columns with the mean
    numeric_cols = all_data.select_dtypes(include=np.number).columns
    all_data[numeric_cols] = all_data[numeric_cols].fillna(all_data[numeric_cols].mean())

    # Scale features
    scaler = StandardScaler()
    # Exclude 'Surname' from feature_cols
    feature_cols = [col for col in all_data.columns if col not in ['id', 'Exited', 'Surname']]
    all_data[feature_cols] = scaler.fit_transform(all_data[feature_cols])

    # Split back into train and test
    train_data = all_data[:len(train_data)]
    test_data = all_data[len(train_data):]

    X = train_data[feature_cols].values
    y = train_data['Exited'].values
    X_test = test_data[feature_cols].values

    return X, y, X_test

In [31]:
# Define cross-validation function
def cross_validate(X, y, knn, n_splits=5):
    # TODO: Implement cross-validation
    # Compute ROC AUC scores

    kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)
    scores = []

    for train_index, val_index in kf.split(X):
        X_train, X_val = X[train_index], X[val_index]
        y_train, y_val = y[train_index], y[val_index]

        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_val)
        score = roc_auc_score(y_val, y_pred)
        scores.append(score)

    return scores

In [32]:
# Load and preprocess data
X, y, X_test = preprocess_data('train.csv', 'test.csv')

# Create and evaluate model
knn = KNN(k=5, distance_metric='euclidean')

# Perform cross-validation
cv_scores = cross_validate(X, y, knn)

print("Cross-validation scores:", cv_scores)
print("Mean ROC AUC score:", np.mean(cv_scores))

# TODO: hyperparamters tuning
k_values = [3, 5, 7, 9, 11]
distance_metrics = ['euclidean', 'manhattan']
best_k = 0
best_metric = ''
best_score = 0

for k in k_values:
    for metric in distance_metrics:
        knn = KNN(k=k, distance_metric=metric)
        scores = cross_validate(X, y, knn)
        mean_score = np.mean(scores)
        if mean_score > best_score:
            best_score = mean_score
            best_k = k
            best_metric = metric

print(f"Best hyperparameters: k={best_k}, distance_metric={best_metric}")
print(f"Best mean ROC AUC score: {best_score}")

# TODO: Train on full dataset with optimal hyperparameters and make predictions on test set
knn = KNN(k=best_k, distance_metric=best_metric)
knn.fit(X, y)
test_predictions = knn.predict(X_test)

# Save test predictions
pd.DataFrame({'id': pd.read_csv('test.csv')['id'], 'Exited': test_predictions}).to_csv('submissions.csv', index=False)

Cross-validation scores: [0.7686115079322456, 0.775914882081865, 0.7654878653573722, 0.7535063969846578, 0.7649943447011512]
Mean ROC AUC score: 0.7657029994114584
Best hyperparameters: k=5, distance_metric=manhattan
Best mean ROC AUC score: 0.7679656739060156
